<div class='alert alert-info' style='text-align: center;'><h1>RSNA-MICCAI Brain Tumor Image Planes</h1></div>
                                                  
#### This notebook gets all the image planes for each series in the train set and appends them to the labels dataframe.

- The same technique can be applied to the test set to ensure coplanar comparissons.
- You can run it yourself, or just D/L the output of this notebook and import into your own.
- Here, we'll get an image from each series and check the plane.

In [1]:
import os
import numpy as np
import pandas as pd
import pydicom
import matplotlib.pyplot as plt

In [2]:
series_names = ['FLAIR','T1w','T1wCE','T2w']
directory = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/'
labels_df = pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv')

In [3]:
# Get an image path from a series, we'll return only the tenth image in the series directory
def get_series_list(study_id, series_name):
 
    files = [os.path.join(directory + '' + study_id + "/" + series_name, f) for f in os.listdir(directory + '' + study_id + "/" + series_name) if 
    os.path.isfile(os.path.join(directory + '' + study_id + "/" + series_name, f))]
    return files[10]

In [4]:
# Convert the Image Orientation Patient tag cosine values into a text string of the plane.
# This is a simple method to represent the plane the image is 'closest to' .. it does not explain any obliqueness.
# There is a more complex (and complete) way to do this, but this is all we need here since all the planes in this dataset are orthogonal.
def get_image_plane(loc):

    row_x = round(loc[0])
    row_y = round(loc[1])
    row_z = round(loc[2])
    col_x = round(loc[3])
    col_y = round(loc[4])
    col_z = round(loc[5])

    if row_x == 1 and row_y == 0 and col_x == 0 and col_y == 0:
        return "Coronal"

    if row_x == 0 and row_y == 1 and col_x == 0 and col_y == 0:
        return "Sagittal"

    if row_x == 1 and row_y == 0 and col_x == 0 and col_y == 1:
        return "Axial"

    return "Unknown"

In [5]:
# Iterate over the labels_df
for index, row in labels_df.iterrows():
    images = []
    study_id = str(row['BraTS21ID']).zfill(5)
    
    # Get a list of one image from each series
    for ser in series_names:
        images.append(get_series_list(study_id, ser))

    # Iterate through the images list and check the plane of each
    for img in images:

        image = pydicom.dcmread(img)
        
        # Get the IOP tag for this image
        image_orientation_patient = image[0x0020,0x0037]
        
        # Get the plane for this image/series
        plane = get_image_plane(image_orientation_patient)

        # Add the plane data to the labels dataframe
        img_temp = img.split("/")
        labels_df.at[index,'plane_' + img_temp[5]] = plane

In [6]:
labels_df.head(20)

,BraTS21ID,MGMT_value,plane_FLAIR,plane_T1w,plane_T1wCE,plane_T2w
0,0,1,Coronal,Axial,Coronal,Sagittal
1,2,1,Sagittal,Axial,Coronal,Sagittal
2,3,0,Sagittal,Axial,Coronal,Sagittal
3,5,1,Coronal,Axial,Coronal,Sagittal
4,6,1,Axial,Axial,Coronal,Sagittal
5,8,1,Sagittal,Axial,Axial,Sagittal
6,9,0,Sagittal,Axial,Coronal,Sagittal
7,11,1,Sagittal,Axial,Coronal,Axial
8,12,1,Coronal,Axial,Axial,Coronal
9,14,1,Coronal,Axial,Axial,Axial


#### We see that most of the columns have different planes. Let's check unique values to see if *any* are in the same plane across the entire dataset.

In [7]:
print("FLAIR: ", labels_df['plane_FLAIR'].unique())
print("T1w: ", labels_df['plane_T1w'].unique())
print("T1wCE: ", labels_df['plane_T1wCE'].unique())
print("T2w: ", labels_df['plane_T2w'].unique())

FLAIR:  ['Coronal' 'Sagittal' 'Axial']
T1w:  ['Axial' 'Coronal' 'Sagittal']
T1wCE:  ['Coronal' 'Axial' 'Sagittal']
T2w:  ['Sagittal' 'Axial' 'Coronal']


#### Now we can see that none of the series are all in the same plane, as they all have *some* series in each plane.

- Export the updated labels dataframe to CSV.

In [8]:
labels_df.to_csv("train_labels_with_planes.csv", index=False)